In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

<h3>Classification</h3>
<h5>Where regression was used to preditic a numeric value, <br>classification ins used to separate data points into classes of different labels.</h5>
<hr color='green'>

In [8]:
#dataset
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "itis_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
)

In [35]:
csv_column_names=[ 'SepalLength', 'SepalWidth', 'PetalLenght', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica'] #0,1,2 in order

In [4]:
train = pd.read_csv(train_path, names=csv_column_names, header=0)
test = pd.read_csv(test_path, names=csv_column_names, header=0)

In [5]:
train.head()

,SepalLength,SepalWidth,PetalLenght,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
test.head()

,SepalLength,SepalWidth,PetalLenght,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


In [9]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [10]:
train_y

0      2
1      1
2      2
3      0
4      0
      ..
115    1
116    1
117    0
118    0
119    1
Name: Species, Length: 120, dtype: int64

<h5 align='center'>Input function</h5>
<hr color='green'>

In [18]:
def input_fn(features, labels, training=True, batch_size=256):
  ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training:
    ds=ds.shuffle(1000).repeat()
  return ds.batch(batch_size)

In [13]:
feature_col =[]
for key in train.keys():
  feature_col.append(tf.feature_column.numeric_column(key=key))
print(feature_col)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLenght', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


<h5 align='center'>Building the Model</h5>
<hr color='green'>

In [16]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_col,
    hidden_units=[30,10],
    n_classes=3)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmptobbek_5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
x = lambda: print('hi')
x()

hi


In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True), 
    steps=5000)

In [27]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-02-03T20:37:48
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmptobbek_5/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.45013s
INFO:tensorflow:Finished evaluation at 2022-02-03-20:37:48
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.6, average_loss = 0.5330712, global_step = 5000, loss = 0.5330712
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmptobbek_5/model.ckpt-5000

Test set accuracy: 0.600



In [38]:
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLenght', 'PetalWidth']
predict={}
print('Please type numeric values as prompted')

for i in features:
  valid = True
  while valid:
    val = input(i + ':')
    if not val.isdigit(): valid=False
  predict[i] = [float(val)]

prediction = classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in prediction:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('\n\nPrediction is {} ({:.1f}%)'.format(SPECIES[class_id], 100*probability))



Please type numeric values as prompted
SepalLength:2.1
SepalWidth:1.2
PetalLenght:4.3
PetalWidth:2.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmptobbek_5/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is Virginica (69.6%)
